In [ ]:
import pandas as pd
import numpy as np
print("the code is running")
# from Package.Models import *
import torch
import pickle
from torch import nn
from torch.nn.modules.padding import ConstantPad1d,ReflectionPad1d
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from sklearn.model_selection import train_test_split
from numba import jit
from matplotlib import pyplot as plt
from torch.autograd import Variable
from sklearn.utils import shuffle
from time import time
from textblob import TextBlob as tb
from nltk.tokenize import TweetTokenizer
import re
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support
tknzr = TweetTokenizer()
print("import statements finished")

DEBUG  = True
torch.manual_seed(2)

def log(*argv):
    if DEBUG == True:
        try:
            print(argv)
        except:
            print("Error in printing")
            
def preprocess(t):    
    t = t.encode("ASCII","ignore").decode("utf-8")
    t = " ".join(tknzr.tokenize(t))
    t = str(t).lower()
    t = re.sub(r'["\'|?"]','',t)
    t = re.sub(r'[\-]',' ',t)
    return t

def get_batches(x,y = 0,chunks = 400):
    l = len(x)
    cnt = int(np.round(int(l)/int(chunks)))
    remain = l%chunks
    rmt = remain/cnt
    x_list = [x[each*cnt:(each*cnt)+cnt] for each in range(chunks+int(rmt)+2)]
    try:
        if y == 0:
            return  x_list

    except:
        y_list = [y[each*cnt:(each*cnt)+cnt] for each in range(chunks+int(rmt)+2)]
        return x_list,y_list        

@jit
def numba_mean(loss_list):
    return np.array(loss_list).mean()

def load_glove_model(gloveFile):
    import numpy as np
    print ("Loading Glove Model")
    f = open(gloveFile,'r',encoding="utf8")
    print("file read")
    model = {}
    count = 0
    for line in f:
        splitLine = line.split()
        word = splitLine[0]
        try:
            embedding = np.array([float(val) for val in splitLine[1:]])
            model[word] = embedding
        except:
            count = count+1
            pass
    print(count)
    print ("Done.",len(model)," words loaded!")
    return model

## loading the glove vectors
# embedding_loc = "../input/embeddings/glove.840B.300d/glove.840B.300d.txt"
embedding_loc = "../input/embeddings/wiki-news-300d-1M/wiki-news-300d-1M.vec"
try:
    glv
except:
    glv = load_glove_model(embedding_loc)
glv[""] = np.zeros((300))
print("trying to load vectors")    
print(len(glv))


In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
dtype = torch.FloatTensor
torch.set_default_tensor_type('torch.FloatTensor')

epoch = 1
lr = 0.0006
checkpoint = 1000
test_checkpoint = 10
threshold = 0.5
batches = 300

## Model definition
class encoder_rnn(nn.Module):
    def __init__(self,input_dim, hidden_dim, layer_dim, output_dim):
        super(encoder_rnn,self).__init__()
        self.rnn = nn.GRU(input_dim, hidden_dim, layer_dim, batch_first=True,bidirectional = True)
        self.layer_dim = layer_dim
        self.hidden_dim = hidden_dim
        self.fc  = nn.Sequential(
                  nn.Linear(hidden_dim*2, hidden_dim*6),
                   nn.BatchNorm1d(num_features = (hidden_dim*6)),
                  nn.ReLU(),
#                 nn.Linear(hidden_dim*6, hidden_dim*4),
#                    nn.BatchNorm1d(num_features = (hidden_dim*4)),
#                   nn.ReLU(),
                  nn.Linear(hidden_dim*6,output_dim),
                  nn.Sigmoid()
                  )
        self.relu = nn.ReLU()
        self.bn1 = nn.BatchNorm1d(num_features = hidden_dim*2)

    def forward(self,x):
        batch_output = torch.zeros(1,self.hidden_dim*2)
        for each in range(len(x)):
            row = x[each]
            for e in range(len(row)):
                X =  (torch.from_numpy(row[e])[None,None,:]).type(dtype)   
                if e == 0:
    #                 h0 = (torch.randn(self.layer_dim*2, 1, self.hidden_dim).type(dtype),torch.randn(self.layer_dim*2, 1, self.hidden_dim).type(dtype))
                    h0 = torch.randn(self.layer_dim*2, 1, self.hidden_dim).type(dtype)
                    output,hidden = self.rnn(X,h0)
                else:
                    output,hidden = self.rnn(X,hidden)
            output = (output).view(-1)
            output = output[None,:]
            batch_output = torch.cat((batch_output,output),0)

        batch_output = (self.bn1(batch_output[1:]))
        final = self.fc(batch_output) 
        return final
    
def init_weights(m):
    if type(m) == nn.Linear:
        print("Xavier applied")
        torch.nn.init.xavier_uniform(m.weight)
        m.bias.data.fill_(0.01)
    
## initiate a model   
model  = encoder_rnn(301,20,1,1)
model.apply(init_weights)

## loading the training data
# q = pd.read_csv("../input/quora-insincere-questions-classification/train.csv",index_col = None,encoding = "iso-8859-1")
q = pd.read_csv("../input/train.csv",index_col = None,encoding = "iso-8859-1")
# q = q[:50000]
print("Considered ....",  str(q.shape))
sin = q[q["target"] == 0]
insin = q[q["target"] == 1]
sin = sin.sample(frac = 0.10,random_state = 42)
q = pd.concat([sin,insin])
q = shuffle(q).reset_index(drop = True)
log("Total rows ",q.shape)

def get_vectors(text,glv,pad = 250,get_pos = False):
    import nltk
    import numpy as np
    glv_text = " ".join([ each for each in text.split(" ") if each in glv]) 
    all_vectors = [glv[each] for each in glv_text.split(" ")]
    all_vectors = [np.append(all_vectors[i],(tb(each).sentiment[0])) for i,each in enumerate(glv_text.split(" "))]
    temp = np.zeros((301))
    if pad != None:
        all_vectors = all_vectors[:pad]
        if len(all_vectors) <pad:
            temp_range = pad-len(all_vectors)
            temp_list = temp.tolist()
            [all_vectors.append(temp_list)for each in range(temp_range)]
    if len(all_vectors)== 0:
        log("The words in this text has no vectors")
        log(text)
        all_vectors.append(temp)
    all_vectors = np.array(all_vectors)
    if get_pos == True:
        pos_vector = get_pos_tag(text,glv)
        if len(all_vectors)!= len(pos_vector):
            print("Pos Error detected")
        all_vectors = [np.concatenate((each,pos_vector[1])) for i,each in enumerate(all_vectors) if len(all_vectors)== len(pos_vector)]
    return all_vectors

log(q.groupby("target")["qid"].count())
log("Considered rows ",q.shape)
q["question_text"] = [ preprocess(each) for each in  q["question_text"]]
log("preprocessing words complete")
q["x"]  = q["question_text"]
q["y"] = q["target"]

## test train split
x,xt,y,yt = train_test_split(q['x'],q['y'],test_size=0.005, random_state=42)
x = x.reset_index(drop = True)
xt = xt.reset_index(drop = True)
y = y.reset_index(drop = True)
yt =yt.reset_index(drop = True)

## readying the vectors for the test data
xt = [get_vectors(each,glv,pad= None) for each in xt]

log("Vectorization complete")

## obtaining batches
x_batches,y_batches = get_batches(x,y,chunks = batches)
x_batches = [each.reset_index(drop = True) for each in x_batches if len(each)!=0]
y_batches = [each.reset_index(drop = True) for each in y_batches if len(each)!=0]

log("splitting the batches is complete")  
log("The shape of each batch is ",x_batches[0].shape)      
log("The shape of test data is ",len(xt))      

def feed(x,y,backprop = False,epoch=1,threshold = 0.5):
    t1 = time()
    loss_list = []
    X = x
    Y = (torch.from_numpy(np.array(y))).type(dtype)
    pred = model.forward(X)
    loss =cross(pred,Y).type(dtype)
    if backprop == True:
        loss.backward()
        optimizer.step()
    loss_list.append(loss.item())
    y_pred = pred.detach().numpy()
    y_pred[y_pred >= threshold] = 1
    y_pred[y_pred < threshold] = 0
    y_pred = y_pred.flatten().tolist()
    y_true = y.tolist()
#     print(precision_recall_fscore_support(y_true = y_true,y_pred = y_pred ,average = "micro" ))
    print(classification_report(y_true = y_true,y_pred = y_pred  ))
    log("T:",str(time()-t1))
    return loss_list


optimizer = torch.optim.Adam(model.parameters(), lr=lr)
optimizer.zero_grad()
cross = nn.BCELoss()

log("Model training starts")

for each in range(epoch):
    epoch_cost = []
    for j,e in enumerate(x_batches):
        x, y = x_batches[j],y_batches[j]
        x = [get_vectors(each,glv,pad=None) for each in x]
        train_cost = numba_mean(feed(x,y,backprop = True,epoch = each,threshold = threshold))  
        epoch_cost.append(train_cost)
        
        log("Epoch num - ",each," and batch num ",j)
        log(train_cost)
        if j%test_checkpoint == 0:
            log("TEST COST")
            test_cost = numba_mean(feed(xt,yt,backprop = False,epoch = each,threshold = threshold))
            log(test_cost)        
        log("================================================")
        if j == checkpoint:
            break
    log("EPOCH ",str(np.mean(epoch_cost)))
#     break
print("completed")


Using device: cpu
Xavier applied
Xavier applied


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:56: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.


Considered .... (1306122, 3)
('Total rows ', (203341, 3))
(target
0    122531
1     80810
Name: qid, dtype: int64,)
('Considered rows ', (203341, 3))
('preprocessing words complete',)
('Vectorization complete',)
('splitting the batches is complete',)
('The shape of each batch is ', (674,))
('The shape of test data is ', 1017)
('Model training starts',)


/opt/conda/lib/python3.6/site-packages/torch/nn/functional.py:2016: UserWarning: Using a target size (torch.Size([674])) that is different to the input size (torch.Size([674, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))


              precision    recall  f1-score   support

           0       0.62      0.69      0.66       413
           1       0.41      0.34      0.38       261

   micro avg       0.56      0.56      0.56       674
   macro avg       0.52      0.52      0.52       674
weighted avg       0.54      0.56      0.55       674

('T:', '6.761310577392578')
('Epoch num - ', 0, ' and batch num ', 0)
(0.7044059038162231,)
('TEST COST',)


...

              precision    recall  f1-score   support

           0       0.88      0.91      0.90       389
           1       0.88      0.84      0.85       285

   micro avg       0.88      0.88      0.88       674
   macro avg       0.88      0.87      0.88       674
weighted avg       0.88      0.88      0.88       674

('T:', '6.841558218002319')
('Epoch num - ', 1, ' and batch num ', 227)
(0.3100900948047638,)
('================================================',)
              precision    recall  f1-score   support

           0       0.87      0.89      0.88       400
           1       0.83      0.80      0.82       274

   micro avg       0.85      0.85      0.85       674
   macro avg       0.85      0.85      0.85       674
weighted avg       0.85      0.85      0.85       674

('T:', '6.661730527877808')
('Epoch num - ', 1, ' and batch num ', 228)
(0.35128656029701233,)
('================================================',)
              precision    recall  f1-score   sup

              precision    recall  f1-score   support

           0       0.89      0.91      0.90       410
           1       0.86      0.83      0.84       264

   micro avg       0.88      0.88      0.88       674
   macro avg       0.87      0.87      0.87       674
weighted avg       0.88      0.88      0.88       674

('T:', '6.932783365249634')
('Epoch num - ', 1, ' and batch num ', 243)
(0.32039862871170044,)
('================================================',)
              precision    recall  f1-score   support

           0       0.88      0.91      0.89       410
           1       0.85      0.80      0.83       264

   micro avg       0.87      0.87      0.87       674
   macro avg       0.86      0.86      0.86       674
weighted avg       0.87      0.87      0.87       674

('T:', '6.89107084274292')
('Epoch num - ', 1, ' and batch num ', 244)
(0.31217160820961,)
('================================================',)
              precision    recall  f1-score   suppor

              precision    recall  f1-score   support

           0       0.88      0.92      0.90       410
           1       0.87      0.80      0.83       264

   micro avg       0.87      0.87      0.87       674
   macro avg       0.87      0.86      0.87       674
weighted avg       0.87      0.87      0.87       674

('T:', '6.818358659744263')
('Epoch num - ', 1, ' and batch num ', 260)
(0.3325806260108948,)
('TEST COST',)
              precision    recall  f1-score   support

           0       0.89      0.90      0.89       625
           1       0.84      0.81      0.83       392

   micro avg       0.87      0.87      0.87      1017
   macro avg       0.86      0.86      0.86      1017
weighted avg       0.87      0.87      0.87      1017

('T:', '4.078894853591919')
(0.30061522126197815,)
('================================================',)
              precision    recall  f1-score   support

           0       0.89      0.95      0.92       395
           1       0.92

              precision    recall  f1-score   support

           0       0.90      0.92      0.91       410
           1       0.87      0.84      0.85       264

   micro avg       0.89      0.89      0.89       674
   macro avg       0.88      0.88      0.88       674
weighted avg       0.89      0.89      0.89       674

('T:', '6.820111513137817')
('Epoch num - ', 1, ' and batch num ', 276)
(0.30492103099823,)
('================================================',)
              precision    recall  f1-score   support

           0       0.91      0.91      0.91       412
           1       0.86      0.86      0.86       262

   micro avg       0.89      0.89      0.89       674
   macro avg       0.89      0.89      0.89       674
weighted avg       0.89      0.89      0.89       674

('T:', '6.7051331996917725')
('Epoch num - ', 1, ' and batch num ', 277)
(0.2914365231990814,)
('================================================',)
              precision    recall  f1-score   suppo

              precision    recall  f1-score   support

           0       0.90      0.91      0.91       405
           1       0.87      0.85      0.86       269

   micro avg       0.89      0.89      0.89       674
   macro avg       0.88      0.88      0.88       674
weighted avg       0.89      0.89      0.89       674

('T:', '6.7887914180755615')
('Epoch num - ', 1, ' and batch num ', 292)
(0.29287460446357727,)
('================================================',)
              precision    recall  f1-score   support

           0       0.91      0.89      0.90       410
           1       0.83      0.87      0.85       264

   micro avg       0.88      0.88      0.88       674
   macro avg       0.87      0.88      0.87       674
weighted avg       0.88      0.88      0.88       674

('T:', '6.8743860721588135')
('Epoch num - ', 1, ' and batch num ', 293)
(0.31362712383270264,)
('================================================',)
              precision    recall  f1-score   

In [ ]:
# test = pd.read_csv("../input/quora-insincere-questions-classification/test.csv",index_col = None)
test = pd.read_csv("../input/test.csv",index_col = None)
log("Test Shape",test.shape)
test["question_text"] = [preprocess(each) for each in test["question_text"]]
log("preprocessing words complete")

test["x"]  = test["question_text"]

total_result = []
tx_batches = get_batches(x = test["x"],chunks = batches)
tx_batches = [each for each in tx_batches if each.shape[0] != 0]
for i,e in enumerate(tx_batches):
    xv = [get_vectors(each,glv,pad = None) for each in e]
    pred =  model.forward(xv)
    result =pred[:,0].detach().numpy()
    [total_result.append(r) for r in result]
    print(i)
test["result"] = total_result
test["prediction"] = [ 1 if each >= threshold else 0 for each in test.result]
test[["qid","prediction"]].to_csv("submission.csv",index = None)
print("submitted")

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
